# Intégration Données Météo & Débit Station P
on veut visualiser le débit smooth de la station P vs la moyenne des précipitations des villes environnantes.

1. Charger le débit propre (minute).
2. Charger les 7 fichiers météo (heure).
3. Calculer une "Météo Moyenne" sur la zone.
4. Ré-échantillonner (Resample) le débit à l'heure pour s'aligner sur la météo.
5. Merger et Visualiser.

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

pd.options.plotting.backend = "plotly"

## 1. Chargement du Débit Nettoyé

In [2]:
file_path_sensor = 'bronze/bronze/sensor_P_clean.csv'
df_flow = pd.read_csv(file_path_sensor)
df_flow['ts'] = pd.to_datetime(df_flow['ts'])
df_flow = df_flow.set_index('ts')

print(f"Période : {df_flow.index.min()} -> {df_flow.index.max()}")
df_flow.head(3)

Période : 2024-05-03 16:23:00 -> 2024-12-31 23:00:00


,entry_debit_f1,entry_debit_clean,entry_debit_smooth_3h
ts,,,
2024-05-03 16:23:00,111.400000,111.400000,NaN
2024-05-03 16:24:00,111.933333,111.933333,NaN
2024-05-03 16:25:00,112.333333,112.333333,NaN


## 2. Chargement et Agrégation de la Météo (7 Villes)

In [3]:
meteo_paths = [
    'bronze/bronze/weather_P/Beaumont-sur-Oise_hourly.csv',
    'bronze/bronze/weather_P/Bernes-sur-Oise_hourly.csv',
    'bronze/bronze/weather_P/Chambly_hourly.csv',
    'bronze/bronze/weather_P/Mours_hourly.csv',
    'bronze/bronze/weather_P/Nointel_hourly.csv',
    'bronze/bronze/weather_P/Persan-Beaumont_hourly.csv',
    'bronze/bronze/weather_P/Ronquerolles_hourly.csv'
]

dfs_meteo = []

for path in meteo_paths:
    try:
        temp_df = pd.read_csv(path)
        temp_df['date'] = pd.to_datetime(temp_df['date'])
        temp_df = temp_df.set_index('date')
        
        filename = os.path.basename(path) 
        city_name = filename.split('_')[0]
        
        # On ne garde que la pluie ('rain') et on la renomme avec le nom de la ville
        if 'rain' in temp_df.columns:
            temp_df = temp_df[['rain']].rename(columns={'rain': city_name})
            dfs_meteo.append(temp_df)
            print(f"  - {city_name} : OK")
        else:
            print(f"Pas de colonne 'rain' dans {filename}")
            
    except FileNotFoundError:
        print(f"Fichier introuvable : {path}")

df_meteo_all = pd.concat(dfs_meteo, axis=1)

df_meteo_all['mean_rain'] = df_meteo_all.mean(axis=1)

df_meteo_all[['mean_rain']].head(3)

  - Beaumont-sur-Oise : OK
  - Bernes-sur-Oise : OK
  - Chambly : OK
  - Mours : OK
  - Nointel : OK
  - Persan-Beaumont : OK
  - Ronquerolles : OK


,mean_rain
date,
2015-01-01 00:00:00,0.0
2015-01-01 01:00:00,0.0
2015-01-01 02:00:00,0.0


## 3. Resampling et Merge (Alignement des fréquences)

La météo est à l'heure (`1H`), le débit est à la minute (`1T`).  
On va passer le débit à l'heure (moyenne) pour aligner les deux datasets.

In [4]:
# On fait la moyenne des valeurs sur chaque heure
df_flow_hourly = df_flow.resample('1H').mean()
df_final = df_flow_hourly.join(df_meteo_all['mean_rain'], how='inner')
df_final = df_final.dropna()

print(f"{df_final.shape[0]} heures de données communes.")
print(f"Période : Du {df_final.index.min()} au {df_final.index.max()}")
df_final.head()

5800 heures de données communes.
Période : Du 2024-05-03 17:00:00 au 2024-12-31 21:00:00


C:\Users\Cecil\AppData\Local\Temp\ipykernel_9312\2406780900.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_flow_hourly = df_flow.resample('1H').mean()


,entry_debit_f1,entry_debit_clean,entry_debit_smooth_3h,mean_rain
2024-05-03 17:00:00,3.065556,7.669444,62.780873,0.000000
2024-05-03 18:00:00,80.517778,80.517778,56.037343,0.000000
2024-05-03 19:00:00,76.610556,76.610556,57.670549,0.028571
2024-05-03 20:00:00,11.859444,16.275000,67.326648,0.100000
2024-05-03 21:00:00,121.600000,121.600000,54.719972,0.000000


## 4. Visualisation : Débit vs Pluie
Vérifions visuellement la corrélation.

In [5]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=df_final.index, y=df_final['entry_debit_smooth_3h'], 
               name="Débit Station (m3/h)", line=dict(color='green', width=2)),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_final.index, y=df_final['mean_rain'], 
               name="Pluie Moyenne Zone (mm)", 
               line=dict(color='blue', width=1), 
               fill='tozeroy', opacity=0.3),
    secondary_y=True,
)

fig.update_layout(
    title_text="Analyse Croisée : Débit Entrant vs Précipitations (Période Commune)",
    xaxis_title="Temps (Heure)"
)

fig.update_yaxes(title_text="Débit (m3/h)", secondary_y=False)
fig.update_yaxes(title_text="Pluie (?)", secondary_y=True)

fig.show()

Question : pourquoi les pics de pluie ne coincident pas exactement avec les pics de débit ? / Il y a des pics fantômes peut etre? (par ex zoomer au 20-27 octobre)